In [1]:
import numpy as np

from pathlib import Path


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



from tensorflow.keras.applications import VGG16
import cv2
import gc

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2025-04-25 14:35:41.157690: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-25 14:35:41.266140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-25 14:35:41.266532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-25 14:35:41.280293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-25 14:35:41.322846: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-25 14:35:41.323856: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
train_dir=Path("Emotions/train")
test_dir=Path("Emotions/test")

This cell saves the paths to the images of each emotion as a value to a key(which is the emotion name)

In [3]:
emotions_train_images_dict={
    'angry': list(train_dir.glob("angry/*.jpg")),
    'disgust':list(train_dir.glob("disgust/*.jpg")),
    'fear':list(train_dir.glob("fear/*.jpg")),
    'happy':list(train_dir.glob("happy/*.jpg")),
    'neutral':list(train_dir.glob("neutral/*.jpg")),
    'sad':list(train_dir.glob("sad/*.jpg")),
    'surprise':list(train_dir.glob("surprise/*.jpg")),
}

In [4]:
emotions_test_images_dict={
    'angry': list(test_dir.glob("angry/*.jpg")),
    'disgust':list(test_dir.glob("disgust/*.jpg")),
    'fear':list(test_dir.glob("fear/*.jpg")),
    'happy':list(test_dir.glob("happy/*.jpg")),
    'neutral':list(test_dir.glob("neutral/*.jpg")),
    'sad':list(test_dir.glob("sad/*.jpg")),
    'surprise':list(test_dir.glob("surprise/*.jpg")),
}

In [5]:
emotions_labels_dict={
    'angry': 0,
    'disgust':1,
    'fear':2,
    'happy':3,
    'neutral':4,
    'sad':5,
    'surprise':6,
}

In [6]:

# Preallocate arrays with float16 dtype (saves 4x memory vs float32)
x_train = np.empty((7000, 224, 224, 3), dtype='float16')
y_train = np.empty((7000,), dtype='uint8')  # Assuming integer labels

#you can add the test data(in our case it's not so important, we can perform small tests manually)
#x_test = np.empty((len(emotions_train_images_dict[emotion_name]), 224, 224, 3), dtype='float16')
#y_test = np.empty((len(emotions_train_images_dict[emotion_name]),), dtype='uint8')


This cell is supposed to fill the x_train and y_train arrays with the preprocessed data

In [7]:
# Fill arrays incrementally
idx = 0
RGB_MEAN = np.array([123.68, 116.779, 103.939], dtype=np.float16)
for emotion_name, images in emotions_train_images_dict.items():
    for image in images[:1070]:
        # Load image as uint8 (OpenCV default)
        img = cv2.imread(str(image))  # Shape: (H, W, 3), dtype: uint8        
            # Resize and convert color
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Still uint8
            
        
        img = img.astype(np.float16)
    
        img -= RGB_MEAN
        x_train[idx] = img
        y_train[idx] = emotions_labels_dict[emotion_name]
        idx += 1
x_train = x_train[:idx]
y_train = y_train[:idx]


In [8]:
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze layers


# Clone model and convert to float16
pretrained_model = tf.keras.models.clone_model(base_model)
pretrained_model.set_weights([w.astype('float16') for w in base_model.get_weights()])

# Set float16 policy for all layers (critical for inference)
for layer in pretrained_model.layers:
    layer._dtype_policy = tf.keras.mixed_precision.Policy('float16')

In [10]:
model = Sequential([
    layers.Input((224,224,3)),
    pretrained_model,          # Add frozen VGGFace
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # 7 emotion classes
])

In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # For integer labels (y_train/y_test)
    metrics=['accuracy']
)

In [12]:
history = model.fit(
    x_train, y_train,
    epochs=5,
)

2025-04-25 14:38:55.857271: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4128079872 exceeds 10% of free system memory.


Epoch 1/5


2025-04-25 14:43:23.846318: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-04-25 14:43:24.208561: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-04-25 14:43:25.503626: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-04-25 14:43:25.611287: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


215/215 [==============================] - 1758s 8s/step - loss: 2.1373 - accuracy: 0.2132
Epoch 2/5
215/215 [==============================] - 1741s 8s/step - loss: 1.7896 - accuracy: 0.2760
Epoch 3/5
215/215 [==============================] - 1916s 9s/step - loss: 1.7382 - accuracy: 0.3044
Epoch 4/5
215/215 [==============================] - 1969s 9s/step - loss: 1.6885 - accuracy: 0.3361
Epoch 5/5
215/215 [==============================] - 1979s 9s/step - loss: 1.6552 - accuracy: 0.3509


This cell is just to test the model (you can replace the string 'happy' with any of the emotions and see whther the model can predict it or not)

In [14]:
RGB_MEAN = np.array([123.68, 116.779, 103.939], dtype=np.float16)
img = cv2.imread(str(emotions_train_images_dict['happy'][0]))  # Shape: (H, W, 3), dtype: uint8        
        # Resize and convert color
img = cv2.resize(img, (224, 224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Still uint8        
img = img.astype(np.float16)


img -= RGB_MEAN
img = np.expand_dims(img, axis=0) 
preds = model.predict(img)
                # Assume preds is a (1, N) array of probabilities
label_index = np.argmax(preds[0])
idx_to_emotion = {idx: name for name, idx in emotions_labels_dict.items()}
emotion_name=idx_to_emotion.get(label_index)
label = emotion_name
print(label)

1/1 [==============================] - 0s 354ms/step
happy


The following cell is to capture frames from the webcam feed them to the model to predict the emotion

In [15]:


VGG16_MEAN_RGB = np.array([123.68, 116.779, 103.939], dtype=np.float16)

def prepare_face(face_bgr):
    # 1. Resize to (222, 224)
    face_resized = cv2.resize(face_bgr, (224, 224))
    # 2. Convert BGR -> RGB
    face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
    # 3. Convert to float32 and subtract VGG16 means
    face_normalized = face_rgb.astype(np.float16) - VGG16_MEAN_RGB
    # 4. Expand dims to add batch axis
    return np.expand_dims(face_normalized, axis=0)

def capture_and_classify(device_index=0, cascade_path='haarcascade_frontalface_default.xml'):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + cascade_path)
    cap = cv2.VideoCapture(device_index)
    if not cap.isOpened():
        print("Cannot open camera"); return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 5)

            for (x, y, w, h) in faces:
                # Extract face Region of interest
                face_bgr = frame[y:y+h, x:x+w]

                # Prepare for VGG16
                face_input = prepare_face(face_bgr)

                # Predict with our model
                preds = model.predict(face_input)
              
                label_index = np.argmax(preds[0])
                idx_to_emotion = {idx: name for name, idx in emotions_labels_dict.items()}
                emotion_name=idx_to_emotion.get(label_index)
                label = emotion_name

                # Draw
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
                cv2.putText(frame, label, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

            cv2.imshow('Emotion VGG16', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    capture_and_classify()


1/1 [==============================] - 0s 309ms/step
